In [11]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
from datetime import *

In [12]:
#an empty list where each element is the day
data=[]
#set the start datetime
a= datetime(day=3,month=3,year=2016)

#Extract data from day 0 to day 15 according to what's needed
for x in range(0,5):
    b = a + timedelta(days=x)
    query = ('https://api.forecast.io/forecast/0df21d512781e37a9c5911805db70e15/51.5085300,-0.1257400,2015-'+b.strftime('%m')+'-'+b.strftime('%d')+'T10:00:00')
    data.append(requests.get(query).json())

/Users/anouchkalettre/Library/Enthought/Canopy_32bit/User/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/Users/anouchkalettre/Library/Enthought/Canopy_32bit/User/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/Users/anouchkalettre/Library/Enthought/Canopy_32bit/User/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: Ins

In [13]:
#From Json format to dataframe : data normalization
df_current=json_normalize(data)

#list comprehension to add a column datetime in UTC datetime format with method 
df_current['utc.datetime']=[datetime.fromtimestamp(x) for x in df_current['currently.time']]
df_current['utc.date']=[datetime.strptime(str(df_current['utc.datetime'][x])[:-9],"%Y-%m-%d").date() for x in range(0,5)]
df_current['utc.time']=[datetime.strptime(str(df_current['utc.datetime'][x])[11:-3],"%M:%S").time() for x in range(0,5)]
  
#df_current['utc.datetime'][0]
#print df_current['utc.date'][0]
print df_current['utc.time'][0]

00:10:00


In [14]:
x = len(df_current['hourly.data'][0])
y = len(df_current['hourly.data'])
w = len(df_current['daily.data'])

keys = ['windSpeed', 'temperature']   #to be defined for the hourly dataframe according to interests

# df_hourly is a List comprehension in three parts : 
# 1)goes through all the days 2)goes through all the hours 3) goes through the 'keys' list created to select only this data
df_hourly = pd.DataFrame([[[df_current['hourly.data'][days][hours][key] for key in keys] for hours in range(x)] for days in range(y)],index= df_current['utc.time'])
df_daily = pd.DataFrame([df_current['daily.data'][days][0] for days in range(w)],index= df_current['utc.date'])

df_daily.head(2) #what df_daily looks like


,apparentTemperatureMax,apparentTemperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,cloudCover,dewPoint,humidity,icon,moonPhase,precipType,...,sunriseTime,sunsetTime,temperatureMax,temperatureMaxTime,temperatureMin,temperatureMinTime,time,visibility,windBearing,windSpeed
utc.date,,,,,,,,,,,,,,,,,,,,,
2015-03-03,43.45,1425391200,31.89,1425423600,0.31,31.59,0.65,partly-cloudy-day,0.43,rain,...,1425365024,1425404655,49.51,1425391200,39.02,1425351600,1425340800,7.56,255,12.48
2015-03-04,50.55,1425484800,28.44,1425448800,0.25,31.58,0.64,partly-cloudy-day,0.46,rain,...,1425451293,1425491160,50.55,1425484800,36.07,1425448800,1425427200,7.80,288,10.37


In [15]:
df_hourly.head(2)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
utc.time,,,,,,,,,,,,,,,,,,,,,
00:10:00,"[12.68, 39.99]","[10.47, 39.86]","[7.39, 39.82]","[5.99, 39.02]","[6.45, 39.9]","[8.45, 40.89]","[11.82, 40.86]","[10.37, 40.72]","[9.15, 41.07]","[12.61, 42.22]",...,"[17.49, 49.51]","[20.75, 47.23]","[15.26, 46.81]","[19.61, 45.8]","[14.74, 43.87]","[14.29, 42.94]","[15.27, 41.83]","[14.74, 41.06]","[13.31, 39.99]","[11.58, 39.11]"
00:10:00,"[12.85, 39.06]","[11.85, 37.94]","[12.75, 38.78]","[11.5, 37.94]","[10.15, 38.78]","[9.58, 37.06]","[10.74, 36.07]","[12.21, 37.02]","[13.47, 38.98]","[16.17, 41.15]",...,"[14.77, 49.04]","[15.82, 49.84]","[12.91, 50.55]","[11.48, 49.52]","[7.93, 48.59]","[6.62, 47.39]","[5.29, 46.38]","[5.9, 44.41]","[6.2, 41.96]","[6.81, 40.67]"
